<a href="https://colab.research.google.com/github/sprksh-crypto/MachineLearningProjects/blob/main/birderModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#This code is to differentiate between 1. Birds (no squirrels/chipmunks) 2. Squirrel/Chipmunk
#3. feeder or tree without squirrels, chipmunks, or birds.
#Mount drive.

from google.colab import drive

#for authorisation
drive.mount('/content/drive')

#Drive files in "/content/drive/My Drive".
!ls "/content/drive/My Drive"

import tensorflow as tf
import numpy as np

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
'1 Research 5_3_23.ipynb'		   buildAndTrainBirdsVsSquirrels.ipynb
 birder					  'Essay Drafts'
 birder1				  'ML Class Code 1 (Flowers).ipynb'
 birderModel				  'ML Lab 1'
 birds-10-eachOf-358.tfrecords		  'ML textbook code 1.ipynb'
 birds-20-eachOf-358.tfrecords		  'Physics Project'
 birdsVsSquirrelsModel			  'Physics Scripts'
 birds-vs-squirrels-train.tfrecords	   Pics
 birds-vs-squirrels-validation.tfrecords   preprocessDefinition.ipynb


In [ ]:
filepath=['/content/drive/My Drive/birds-20-eachOf-358.tfrecords']
filepath1=['/content/drive/My Drive/birds-10-eachOf-358.tfrecords']

raw_birdType_training_dataset=tf.data.TFRecordDataset(filepath)
raw_birdType_validation_dataset=tf.data.TFRecordDataset(filepath1)

In [ ]:
#Need to create testing set from given sets.
datLen=raw_birdType_validation_dataset.reduce(0,lambda x,y: x+1)

n_train=int(datLen.numpy()*.7)
n_test=int(datLen.numpy()*.3)

In [ ]:
feature_description={'image':tf.io.FixedLenFeature([],tf.string),'birdType':tf.io.FixedLenFeature([],tf.int64)}

#Define parse example to load data.

def parse_examples(serialized_examples):
  examples=tf.io.parse_example(serialized_examples,feature_description)
  targets=examples.pop('birdType')
  images=tf.image.resize_with_pad(tf.cast(tf.io.decode_jpeg(examples['image'],channels=3),tf.float32),299,299)
  return images, targets

In [ ]:
#Load in dataset.
#But do I want to load it as supervised.
training_dataset_BT=raw_birdType_training_dataset.take(n_train).map(parse_examples,num_parallel_calls=tf.data.AUTOTUNE)
validation_dataset_BT=raw_birdType_validation_dataset.map(parse_examples,num_parallel_calls=tf.data.AUTOTUNE)

testing_dataset_BT=raw_birdType_validation_dataset.skip(n_train).take(n_test).map(parse_examples,num_parallel_calls=tf.data.AUTOTUNE)

In [ ]:
#Preprocessing layer.

def preproc(image,label):
  inp=tf.keras.applications.xception.preprocess_input(image)
  return inp,label

In [ ]:
#Apply preprocessing layer.

training_dataset_BT1=training_dataset_BT.take(n_train).map(preproc,num_parallel_calls=tf.data.AUTOTUNE).batch(64)
validation_dataset_BT1=validation_dataset_BT.map(preproc,num_parallel_calls=tf.data.AUTOTUNE).batch(64)

testing_dataset_BT1=testing_dataset_BT.skip(n_train).take(n_test).map(preproc,num_parallel_calls=tf.data.AUTOTUNE).batch(64)

In [ ]:
#Defining model.
#Remove top layer and look.

base_model=tf.keras.applications.xception.Xception(weights='imagenet',include_top=False)
#base_model.summary()

In [ ]:
#Add new top layer.
avg=tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
output=tf.keras.layers.Dense(358,activation="softmax")(avg)
model=tf.keras.models.Model(inputs=base_model.input,outputs=output)
#model.summary()

In [ ]:
#Freeze the weights of the lower layers.
#Why when print, the last two are true? SHouldn't it just be the last?

for layer in base_model.layers:
  layer.trainable=True

#for layer in model.layers:
#  print(layer.trainable)

In [ ]:
top5err=tf.keras.metrics.SparseTopKCategoricalAccuracy(k=5,name='top5')
top10err=tf.keras.metrics.SparseTopKCategoricalAccuracy(k=10,name='top10')
top20err=tf.keras.metrics.SparseTopKCategoricalAccuracy(k=20,name='top20')

In [ ]:
#Fit the new upper layer.

checkpoint_cb=tf.keras.callbacks.ModelCheckpoint('BirdSquirrelMod-topFit.h5',
save_best_only=True)

earlyStop_cb=tf.keras.callbacks.EarlyStopping(patience=10,restore_best_weights=True)

ss=0.8

opt=tf.keras.optimizers.SGD(learning_rate=ss)

model.compile(loss="sparse_categorical_crossentropy",optimizer=opt,metrics=['accuracy',top5err,top10err,top20err])

model.fit(training_dataset_BT1,validation_data=validation_dataset_BT1,epochs=30,callbacks=[checkpoint_cb,earlyStop_cb])

Epoch 1/30


ResourceExhaustedError: ignored

In [ ]:
model.save('/content/drive/My Drive/birder2')

In [ ]:
#Unfreeze lower layers?